<a href="https://colab.research.google.com/github/abhiiyer/EIP3/blob/master/Assignment-6/Session2_Phase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import string
import requests

Using TensorFlow backend.


In [3]:
link = "http://www.gutenberg.org/cache/epub/11/pg11.txt"
r = requests.get(link, allow_redirects=True)
open('wonderland.txt', 'wb').write(r.content)

167518

In [4]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

raw_text = raw_text.translate(str.maketrans('','',string.punctuation))
raw_text = raw_text.translate(str.maketrans("\n",' ',string.punctuation))
raw_text = raw_text[1:]

# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print( "Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)


Total Characters:  154860
Total Vocab:  37


In [5]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 50
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  154810


In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [7]:
# define the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(128))
model.add(Dense(y.shape[1], activation='softmax'))

model.summary()

W0817 14:16:26.009867 140036626319232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0817 14:16:26.055057 140036626319232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0817 14:16:26.062661 140036626319232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0817 14:16:26.412914 140036626319232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0817 14:16:26.427126 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50, 128)           66560     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 37)                4773      
Total params: 202,917
Trainable params: 202,917
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=100, batch_size=256, callbacks=callbacks_list, verbose=1)

W0817 14:16:26.763658 140036626319232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0817 14:16:26.805314 140036626319232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0817 14:16:26.987336 140036626319232 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
154810/154810 [==============================] - 131s 847us/step - loss: 2.7842

Epoch 00001: loss improved from inf to 2.78422, saving model to weights-improvement-01-2.7842-bigger.hdf5
Epoch 2/100
154810/154810 [==============================] - 126s 814us/step - loss: 2.5472

Epoch 00002: loss improved from 2.78422 to 2.54717, saving model to weights-improvement-02-2.5472-bigger.hdf5
Epoch 3/100
154810/154810 [==============================] - 125s 811us/step - loss: 2.4432

Epoch 00003: loss improved from 2.54717 to 2.44322, saving model to weights-improvement-03-2.4432-bigger.hdf5
Epoch 4/100
154810/154810 [==============================] - 125s 810us/step - loss: 2.3553

Epoch 00004: loss improved from 2.44322 to 2.35529, saving model to weights-improvement-04-2.3553-bigger.hdf5
Epoch 5/100
154810/154810 [==============================] - 125s 805us/step - loss: 2.2706

Epoch 00005: loss improved from 2.35529 to 2.27058, saving model to weights-improvement-05-2.2706-b

In [10]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(500):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" so much about a whiting before  i can tell you mor "
e said alice  i dont know it would not could not look att it said the dat and the three hardeners of the orher way she king harpen she had been looking at the hrrpe and she could not it would be growing and then the dormouse said the duchess and the doumd see it was and the three gardeners of the orosr of the words          how drtm both the ware stiel so she sro counte the words  wes aeginning to she said to herself i should think it was a doyr and she thought alice and he there was a little sa
Done.
